In [12]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
import gc
from sklearn.model_selection import train_test_split
import tensorflow
import gensim
import numpy as np
%matplotlib inline
from subprocess import check_output
print(check_output(["ls", "input/"]).decode("utf8"))

aisles.csv
departments.csv
order_products__prior.csv
order_products__train.csv
orders.csv
prediction_lgbm.pkl
products.csv
sample_submission.csv



In [4]:
pd.set_option('display.max_columns',60)

In [15]:
path = '/Users/shaolongchen/Data Science/Kaggle/Instacart/input/'

In [16]:
aisles = pd.read_csv(path+"aisles.csv")
products = pd.read_csv(path+"products.csv")
departments = pd.read_csv(path+"departments.csv")
#orders = pd.read_csv(path+"orders_membership.csv",index_col=0)
orders = pd.read_csv(path+"orders.csv")
dtrain = pd.read_csv(path+"order_products__train.csv")
dprior = pd.read_csv(path+"order_products__prior.csv")#, nrows=1000000)
sample_submission = pd.read_csv(path + "sample_submission.csv")

In [17]:
print "ailes = (%10d,%3d)\t"%aisles.shape, aisles.columns
print "\nproducts = (%10d,%3d)\t"%products.shape, products.columns
print "\ndepartments = (%10d,%3d)\t"%departments.shape,departments.columns
print "\norders = ",orders.shape, "\t\t", orders.columns
print "\ndtrain = ",dtrain.shape, "\t\t", dtrain.columns
print "\ndprior",dprior.shape, "\t\t", dprior.columns

ailes = (       134,  2)	Index([u'aisle_id', u'aisle'], dtype='object')

products = (     49688,  4)	Index([u'product_id', u'product_name', u'aisle_id', u'department_id'], dtype='object')

departments = (        21,  2)	Index([u'department_id', u'department'], dtype='object')

orders =  (3421083, 7) 		Index([u'order_id', u'user_id', u'eval_set', u'order_number', u'order_dow',
       u'order_hour_of_day', u'days_since_prior_order'],
      dtype='object')

dtrain =  (1384617, 4) 		Index([u'order_id', u'product_id', u'add_to_cart_order', u'reordered'], dtype='object')

dprior (32434489, 4) 		Index([u'order_id', u'product_id', u'add_to_cart_order', u'reordered'], dtype='object')


In [18]:
'''%%time
#calculate the membership days for each order

orders.days_since_prior_order = orders.days_since_prior_order.fillna(0).astype(int)
orders["_user_current_membership_days"] = [0]*(orders.shape[0])

userid_temp = 0
k=0
for orders_index in orders.index:
    if orders.user_id[orders_index] == userid_temp:
        orders.loc[orders_index,"_user_current_membership_days"] = orders.loc[orders_index-1,"_user_current_membership_days"] + orders.loc[orders_index,"days_since_prior_order"]
    else:
        userid_temp = orders.loc[orders_index,"user_id"]
#    k = k+1
#    if k==1000: break
orders.to_csv("orders_temp1.csv")
'''

'%%time\n#calculate the membership days for each order\n\norders.days_since_prior_order = orders.days_since_prior_order.fillna(0).astype(int)\norders["_user_current_membership_days"] = [0]*(orders.shape[0])\n\nuserid_temp = 0\nk=0\nfor orders_index in orders.index:\n    if orders.user_id[orders_index] == userid_temp:\n        orders.loc[orders_index,"_user_current_membership_days"] = orders.loc[orders_index-1,"_user_current_membership_days"] + orders.loc[orders_index,"days_since_prior_order"]\n    else:\n        userid_temp = orders.loc[orders_index,"user_id"]\n#    k = k+1\n#    if k==1000: break\norders.to_csv("orders_temp1.csv")\n'

In [19]:
orders.head(5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [20]:
prior_details = dprior.merge(orders[orders.eval_set == "prior"],how="left",on="order_id")

In [21]:
prior_details.head(2)

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,202279,prior,3,5,9,8.0
1,2,28985,2,1,202279,prior,3,5,9,8.0


In [22]:
def add_groupby_features(df, group_cols, agg_dict):
#    print "grouping in process..."
    grouped = df.groupby(group_cols).agg(agg_dict)
    grouped.columns = grouped.columns.droplevel(0)
    grouped.reset_index(inplace=True)
#    print "group done."
    return grouped

In [23]:
def ka_add_groupby_features_n_vs_1(df, group_columns_list, target_columns_list, methods_list):
        grouped_name = ''.join(group_columns_list)
        target_name = ''.join(target_columns_list)
        combine_name = [[grouped_name] + [method_name] + [target_name] for method_name in methods_list]

        grouped = df.groupby(group_columns_list)

        the_stats = grouped[target_name].agg(methods_list).reset_index()
        the_stats.columns = [grouped_name] + \
                            ['_%s_%s_by_%s' % (grouped_name, method_name, target_name) \
                             for (grouped_name, method_name, target_name) in combine_name]
        return the_stats

# Create Product Features

In [24]:
#####?????? check "sum" for nan values
#Which times does the user buy the product in an order?
prior_details["_userprod_nth_time"] = prior_details.groupby(["user_id","product_id"]).cumcount() + 1
agg_dict1 = {"user_id":{"_prod_total_purchase_cnts":"count"},
         "reordered":{"_prod_reordered_total_cnts":"sum"},
        "_userprod_nth_time":{"_prod_first_purchase_total_cnts":lambda x: sum(x==1),
                                  "_prod_second_purchase_total_cnts":lambda x: sum(x==2)}}
#                                  "_prod_multiple_purchase_total_cnts":lambda x: sum(x>1)}}
product_features = add_groupby_features(prior_details,["product_id"],agg_dict1)

/Users/shaolongchen/anaconda/lib/python2.7/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [25]:
prior_details.head(2)

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,_userprod_nth_time
0,2,33120,1,1,202279,prior,3,5,9,8.0,1
1,2,28985,2,1,202279,prior,3,5,9,8.0,1


In [26]:
agg_dict12 = {'order_dow':{'_prod_average_dow':'mean', '_prod_frequent_dow':lambda x: x.value_counts().idxmax()}, 
            'order_hour_of_day':{'_prod_average_hour':'mean', '_prod_frequent_hour':lambda x: x.value_counts().idxmax()}} 

prod_temp = add_groupby_features(prior_details, ['product_id'], agg_dict12)
product_features = product_features.merge(prod_temp,on="product_id",how="left")

In [27]:
agg_dict15 = {'user_id':{'_prod_distinct_users':lambda x:x.nunique()},
             'add_to_cart_order':{'_prod_mean_cart_position':'mean'}} 

prod_temp = add_groupby_features(prior_details, ['product_id'], agg_dict15)
product_features = product_features.merge(prod_temp,on="product_id",how="left")

In [28]:
'''#train Word2Vec models
dtrain_temp=dtrain.copy()
dprior_temp=dprior.copy()

dtrain_temp["product_id"] = dtrain_temp["product_id"].astype(str)
dprior_temp["product_id"] = dprior_temp["product_id"].astype(str)
train_sentences = dtrain_temp.groupby("order_id").apply(lambda x: x["product_id"].tolist())
prior_sentences = dprior_temp.groupby("order_id").apply(lambda x: x["product_id"].tolist())
sentences = train_sentences.append(prior_sentences)
longest = np.max(sentences.apply(len))
sentences = sentences.values
model = gensim.models.Word2Vec(sentences, size=100, window=longest, min_count=2, workers=7)

del sentences,dtrain_temp,dprior_temp,train_sentences,prior_sentences
gc.collect()
'''

'#train Word2Vec models\ndtrain_temp=dtrain.copy()\ndprior_temp=dprior.copy()\n\ndtrain_temp["product_id"] = dtrain_temp["product_id"].astype(str)\ndprior_temp["product_id"] = dprior_temp["product_id"].astype(str)\ntrain_sentences = dtrain_temp.groupby("order_id").apply(lambda x: x["product_id"].tolist())\nprior_sentences = dprior_temp.groupby("order_id").apply(lambda x: x["product_id"].tolist())\nsentences = train_sentences.append(prior_sentences)\nlongest = np.max(sentences.apply(len))\nsentences = sentences.values\nmodel = gensim.models.Word2Vec(sentences, size=100, window=longest, min_count=2, workers=7)\n\ndel sentences,dtrain_temp,dprior_temp,train_sentences,prior_sentences\ngc.collect()\n'

In [29]:
#add word vectors using product_id
model = gensim.models.Word2Vec.load("product2vec.model") #load already trained model
vocab = list(model.wv.vocab.keys())
word_vectors = model.wv.syn0
wv_sum = np.sum(word_vectors, axis=1)
df_wv = pd.DataFrame({"product_id":[int(x) for x in vocab], "_prod_wv":wv_sum})
df_wv.head(5)

IOError: [Errno 2] No such file or directory: 'product2vec.model'

In [ ]:
product_features = product_features.merge(df_wv,on="product_id",how="left")

In [ ]:
prod_misc = products.merge(aisles,how="left",on="aisle_id").merge(departments, how="left",on="department_id")

In [ ]:
prod_misc.head(2)

In [ ]:
prod_misc["_is_organic"] = [int("Organic" in x) for x in prod_misc.product_name]
prod_misc["_is_banana"] = [int("banana" in x or "Banana" in x) for x in products.product_name]
prod_misc["_is_sweet"] = [int("sweet" in x or "Sweet" in x) for x in products.product_name]
prod_misc["_is_fruit"] = [int("fruit" in x or "Fruit" in x) for x in products.product_name]
prod_misc["_is_fresh"] = [int("fresh" in x or "Fresh" in x) for x in products.product_name]

In [ ]:
prod_misc.head(2)

In [ ]:
product_features = product_features.merge(prod_misc,on="product_id",how="left")

In [ ]:
product_features['_prod_reorder_prob'] = product_features._prod_second_purchase_total_cnts / product_features._prod_first_purchase_total_cnts
product_features['_prod_reorder_ratio'] = product_features._prod_reordered_total_cnts / product_features._prod_total_purchase_cnts
product_features['_prod_reorder_times'] = 1 + product_features._prod_reordered_total_cnts / product_features._prod_first_purchase_total_cnts
# 1 - _prod_reorder_ratio = 1/_prod_reorder_times

In [ ]:
product_features.head(2)

In [ ]:
#del prod_misc,model,df_wv,prod_temp,aisles,departments,products
gc.collect()

In [ ]:
product_features.to_csv("product_features.csv",index=False)

# Create User Features

In [ ]:
agg_dict2 = {"order_number":{"_user_total_orders":"max"},
            "days_since_prior_order":{"_user_membership_days":"sum","_user_mean_order_period":"mean"}}

user_features = add_groupby_features(orders[orders.eval_set=="prior"],["user_id"],agg_dict2)

In [ ]:
agg_dict3 = {"reordered":{"_user_reorder_ratio":lambda x:sum(x)/float(len(x))}}
user_temp = add_groupby_features(prior_details[prior_details.order_number>1],["user_id"],agg_dict3)
user_features = user_features.merge(user_temp,on="user_id",how="left")

In [ ]:
agg_dict4 = {"product_id":{"_user_total_products":'count',
                          "_user_distinct_products": lambda x: x.nunique()}}
user_temp = add_groupby_features(prior_details,["user_id"],agg_dict4)
user_features = user_features.merge(right=user_temp,on="user_id",how="left")

In [ ]:
agg_dict5 = {'product_id':{'each_order_cnts':'count'}}
user_temp = add_groupby_features(prior_details,["order_id"],agg_dict5)
user_temp = user_temp.merge(right=orders[orders.eval_set=="prior"][["order_id","user_id"]],how="left",on="order_id")


agg_dict6 = {'each_order_cnts':{'_user_mean_basket':'mean',
                                 '_user_median_basket':'median',
                                 '_user_min_basket':'min', 
                                 "_user_max_basket":'max', }}
user_temp = add_groupby_features(user_temp,["user_id"],agg_dict6)
user_features = user_features.merge(right=user_temp,on="user_id",how="left")

In [ ]:
user_temp = orders[orders.eval_set != "prior"]
user_features = user_features.merge(right=user_temp,on="user_id",how="left")

In [ ]:
agg_dict13 = {'order_dow':{'_user_average_dow':'mean', '_user_frequent_dow':lambda x: x.value_counts().idxmax()}, 
            'order_hour_of_day':{'_user_average_hour':'mean', '_user_frequent_hour':lambda x: x.value_counts().idxmax()}} 

user_temp= add_groupby_features(prior_details, ['user_id'], agg_dict13)
user_features = user_features.merge(right=user_temp,on="user_id",how="left")

In [ ]:
del user_temp
gc.collect()

In [ ]:
user_features.head(2)

In [ ]:
user_features.drop(["order_number"],axis=1,inplace=True)

In [ ]:
user_features.to_csv("user_features.csv")

# Create User_Products Features

In [ ]:
agg_dict7 = {'order_number':{'_userprod_order_total_cnts': 'count', 
                              '_userprod_first_order_number': 'min', 
                              '_userprod_last_order_number':'max'}, 
              'add_to_cart_order':{'_userprod_average_cart_position': 'mean'},
            '_user_current_membership_days':{'_userprod_first_order_membership':'min',
                                            '_userprod_last_order_membership':'max'}}

userprod_features = add_groupby_features(prior_details, ['user_id', 'product_id'], agg_dict7)

In [ ]:
userprod_features['_userprod_total_days'] = userprod_features._userprod_last_order_membership-userprod_features._userprod_first_order_membership
userprod_features['_userprod_average_span_days'] = 1.0*userprod_features._userprod_total_days/userprod_features._userprod_order_total_cnts

In [ ]:
'''
#This function changes [a,b,c] to [a,a+b,a+b+c]
def sum_list(x_list):
    sum_list = []
    sum_temp=0
    for x in x_list:
        sum_temp = int(np.nansum([x,sum_temp]))
        sum_list = sum_list +[sum_temp]
    return sum_list

test_list = [2,np.nan,1,2,3,4,5,6]
y = sum_list(test_list)
print y

agg_dict14 = {"days_since_prior_order":{_user_current_membership_days: lambda x:sum_list(x)}}
temp = add_groupby_features(orders,['user_id'],agg_dict14)
orders = orders.merge(temp,on="user_id")
orders.head()

for user_id in orders.user_id.unique():
    days_list = list(orders[orders.user_id==user_id].days_since_prior_order)
    user_list = sum_list(days_list)
    _user_current_membership_days = _user_current_membership_days + user_list
df=pd.DataFrame({"order_id":orders.user_id, "_user_current_membership_days": _user_current_membership_days})
orders = orders.merge(df,on="user_id",how="left")
orders[orders.user_id==1]



'''

In [ ]:
orders[orders.user_id==2].index

In [ ]:
orders.shape[0]

In [ ]:
orders["_user_current_membership_days"]=[0]*orders.shape[0]

In [ ]:
orders.loc[2,"order_hour_of_day"]

In [ ]:
userprod_features.to_csv("userprod_features.csv")
userprod_features.head(2)

# Combine Features and Create More Features

In [ ]:
data = userprod_features.merge(product_features, how='inner', on='product_id').merge(user_features, how='inner', on='user_id')

In [ ]:
data['_userprod_order_rate'] = data._userprod_order_total_cnts / data._user_total_orders
data['_userprod_order_cnt_since_last_order'] = data._user_total_orders - data._userprod_last_order_number
data['_userprod_order_rate_since_first_order'] = data._userprod_order_total_cnts / (data._user_total_orders - data._userprod_first_order_number + 1)

data["_user_match_dow"] = abs(np.array(data._user_frequent_dow) - np.array(data.order_dow))
data["_user_match_hour"] = abs(np.array(data._user_frequent_hour) - np.array(data.order_hour_of_day))
data["_prod_match_hour"] = abs(np.array(data._prod_frequent_hour) - np.array(data.order_hour_of_day))
data["_prod_match_dow"] = abs(np.array(data._prod_frequent_dow) - np.array(data.order_dow))

#data['_user_traintest_membership_total_days'] = data._user_membership_days + data.days_since_prior_order
#data['_userprod_days_since_last_order'] = data._user_traintest_membership_total_days - data._userprod_last_order_membership

data['_userprod_days_since_last_order'] = data._user_current_membership_days - data._userprod_last_order_membership

In [ ]:
# add user_id to train set
dtrain = dtrain.merge(right=orders[['order_id', 'user_id']], how='left', on='order_id')

In [ ]:
data = data.merge(dtrain[['user_id', 'product_id', 'reordered']], on=['user_id', 'product_id'], how='left')

In [ ]:
print data.shape
data.head(5)

In [ ]:
np.nansum(data.reordered),np.nansum(dtrain.reordered) #should be equal

In [ ]:
#del prior_details,dtrain,dprior,orders
gc.collect()

In [ ]:
#data.to_csv("data.csv")

# XGB

In [ ]:
X_test = data.loc[data.eval_set == "test",:]
d_test = xgb.DMatrix(X_test.drop(['reordered',"eval_set","user_id","product_id","order_id","aisle_id","department_id","product_name","aisle","department"], axis=1))

In [ ]:
train = data[data.eval_set == "train"]
train['reordered'] = train.reordered.fillna(0)

In [ ]:
#del data
gc.collect()

In [ ]:
train.head(2)

In [ ]:
print sum(train.reordered), train.shape

In [ ]:
# subsample 让training时间更短
X_train, X_val, y_train, y_val = train_test_split(train.drop(['reordered',"eval_set","user_id","product_id","order_id","aisle_id","department_id","product_name","aisle","department"], axis=1), train.reordered,
                                                    test_size=0, random_state=420) #['eval_set','reordered', 'product_id','user_id', 'order_id']
print "splitting done"
#del train
gc.collect()

d_train = xgb.DMatrix(X_train, y_train)
d_val = xgb.DMatrix(X_val,y_val)
print "DMatrix done"

In [ ]:
%%time

xgb_params = {
    "objective"         : "reg:logistic"
    ,"eval_metric"      : "logloss"
    ,"eta"              : 0.1
    ,"max_depth"        : 6
    ,"min_child_weight" :10
    ,"gamma"            :0.70
    ,"subsample"        :0.76
    ,"colsample_bytree" :0.95
    ,"alpha"            :2e-05
    ,"lambda"           :10
}

watchlist= [(d_train, "train")]#,(d_val,'val')]
bst = xgb.train(params=xgb_params, dtrain=d_train, num_boost_round=1000, early_stopping_rounds=10, evals=watchlist, verbose_eval=10)

bst.save_model("bst23.model")

# Choose Threshhold p-value to Maximize F1-Score

In [ ]:
#Find the threshhold p to get best F1-score

from sklearn.metrics import f1_score
p_list = np.linspace(0,0.5,num=101)
score_list = []
for p in p_list:
    score = f1_score(y_true=y_train, y_pred=(bst.predict(d_train) > p).astype(int))
    score_list = score_list+[score]
plt.plot(p_list,score_list)
plt.show()
print "p value for max F1 score:", p_list[np.argmax(score_list)]
print "Maximum F1 score:", max(score_list)
best_p = p_list[np.argmax(score_list)]

In [ ]:
bst.save_model("model_3.model")
bst.dump_model('model_3.txt')
#xgb.plot_importance(bst)
ax = xgb.plot_importance(bst)
fig = ax.figure
fig.set_size_inches(9, 10)

In [ ]:
best_p=0.21
X_test.loc[:,'reordered'] = (bst.predict(d_test) > best_p).astype(int)
X_test.loc[:, 'product_id'] = X_test.product_id.astype(str)#to make str joining convenient

In [ ]:
X_test.head(2)

In [ ]:
submit = ka_add_groupby_features_n_vs_1(X_test[X_test.reordered == 1], 
                                               group_columns_list=['order_id'],
                                               target_columns_list= ['product_id'],
                                               methods_list=[lambda x: ' '.join(list(x))])
submit.head(2)

In [ ]:
#change column names
submit.columns = sample_submission.columns.tolist()
submit_final = sample_submission[['order_id']].merge(submit, how='left').fillna('None')
submit_final.head()

In [ ]:
print sum(submit_final.products=="None"),len(submit_final.products)

In [ ]:
submit_final.to_csv("20170813_trial 1_1000trees_100%data.csv", index=False)